In [1]:
from sklearn.decomposition import KernelPCA
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU
from keras.optimizers import Adam

In [2]:
path = "./UJIIndoorLoc/children_13/sorted/00_sorted.csv"
train_df = pd.read_csv(path,header=0)
print(train_df.shape)
a = train_df.mean()
for i in train_df.columns[:520]:
    if a[i]==-110:
        del train_df[i]
print(train_df.shape)
train_df["REF"] = pd.factorize(train_df["REF"])[0].astype(int)#将标签映射到顺序数字上
labels = train_df.REF.values
features = train_df.drop(columns=['TIMESTAMP','PHONEID','USERID','RELATIVEPOSITION',
                                'SPACEID','BUILDINGID','FLOOR','LATITUDE','LONGITUDE',
                                'BF','REF']).values
features,labels,features.shape

(1059, 531)
(1059, 130)


(array([[-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        ...,
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110],
        [-110, -110, -110, ..., -110, -110, -110]], dtype=int64),
 array([ 0,  0,  0, ..., 53, 53, 53]),
 (1059, 119))

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.3,
                                                    random_state= 150,
                                                    #random_state：可以理解为随机数种子，主要是为了复现结果而设置
                                                    stratify=labels)
X_train,Y_train = shuffle(X_train, Y_train)

In [4]:
scale = StandardScaler().fit(X_train)
X_train_transform = scale.transform(X_train)
X_test_transform = scale.transform(X_test)
Y_train_encoded = to_categorical(Y_train)
Y_test_encoded = to_categorical(Y_test)

In [29]:
rbf_pca=KernelPCA(n_components = 100,kernel='sigmoid',gamma=0.041111).fit(X_train_transform)
X_train_pca = rbf_pca.transform(X_train_transform)
X_test_pca = rbf_pca.transform(X_test_transform)
X_train_pca,X_test_pca,X_train_pca.shape,X_test_pca.shape

(array([[-4.34535880e-01, -4.60094318e-02,  2.27848448e-01, ...,
         -1.60244339e-02, -9.80403083e-03,  7.17231004e-03],
        [-4.26721293e-01, -1.73598267e-01, -2.26627874e-01, ...,
          4.48780299e-02, -3.72480244e-02, -4.53463804e-02],
        [-7.14256665e-01, -1.40135691e-01,  3.05642139e-01, ...,
          2.31420585e-02,  7.35797572e-03, -1.33659276e-02],
        ...,
        [ 4.21523484e-01, -3.41372192e-02, -5.47472967e-02, ...,
          2.36029902e-03, -4.73378493e-03, -3.91495760e-04],
        [ 3.73970629e-01,  1.02227364e-01,  8.38252695e-02, ...,
          4.77197397e-04,  2.22933531e-02,  4.77803264e-03],
        [ 6.49708293e-01,  2.84623747e-01, -5.82309744e-02, ...,
         -2.58836071e-02,  1.32880055e-02, -1.00329626e-02]]),
 array([[-4.24684889e-01, -5.85445519e-02, -8.36540411e-02, ...,
          8.12490341e-03, -9.74926754e-03,  7.10114976e-03],
        [-7.68971887e-01,  3.14778465e-01,  1.32664290e-01, ...,
         -4.08508569e-02, -1.05656496e

### 找出kpca最合适的kernel和gama

In [27]:
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
])
param_grid = [{
        "kpca__gamma": np.linspace(0.01, 0.05,10),
        "kpca__kernel": ["rbf", "sigmoid","cosine","poly"]#不能加上"precomputed"，不然会报错 error: X should be a square kernel matrix
    }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X_train_transform, Y_train)

C:\Users\xfp\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\xfp\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kpca', KernelPCA(n_components=2)),
                                       ('log_reg', LogisticRegression())]),
             param_grid=[{'kpca__gamma': array([0.01      , 0.01444444, 0.01888889, 0.02333333, 0.02777778,
       0.03222222, 0.03666667, 0.04111111, 0.04555556, 0.05      ]),
                          'kpca__kernel': ['rbf', 'sigmoid', 'cosine',
                                           'poly']}])

In [28]:
print(grid_search.best_params_)

{'kpca__gamma': 0.04111111111111111, 'kpca__kernel': 'sigmoid'}


### train

In [35]:
model = Sequential()
model.add(Dense(256, input_shape=(100,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(54, activation='softmax'))
model.compile(optimizer=Adam(0.0002, 0.5), loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train_pca,Y_train_encoded,epochs=250,batch_size=32,validation_split=0.2,verbose=1)

Epoch 1/250
19/19 [==============================] - 1s 12ms/step - loss: 3.9842 - accuracy: 0.0439 - val_loss: 3.9752 - val_accuracy: 0.0537
Epoch 2/250
19/19 [==============================] - 0s 3ms/step - loss: 3.9663 - accuracy: 0.0861 - val_loss: 3.9632 - val_accuracy: 0.0805
Epoch 3/250
19/19 [==============================] - 0s 3ms/step - loss: 3.9460 - accuracy: 0.1064 - val_loss: 3.9477 - val_accuracy: 0.0738
Epoch 4/250
19/19 [==============================] - 0s 4ms/step - loss: 3.9183 - accuracy: 0.1166 - val_loss: 3.9246 - val_accuracy: 0.0872
Epoch 5/250
19/19 [==============================] - 0s 4ms/step - loss: 3.8801 - accuracy: 0.1301 - val_loss: 3.8907 - val_accuracy: 0.0805
Epoch 6/250
19/19 [==============================] - 0s 4ms/step - loss: 3.8261 - accuracy: 0.1385 - val_loss: 3.8411 - val_accuracy: 0.0805
Epoch 7/250
19/19 [==============================] - 0s 5ms/step - loss: 3.7508 - accuracy: 0.1132 - val_loss: 3.7732 - val_accuracy: 0.0604
Epoch 8/250


Epoch 59/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8906 - accuracy: 0.7770 - val_loss: 1.4821 - val_accuracy: 0.5302
Epoch 60/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8714 - accuracy: 0.7686 - val_loss: 1.4762 - val_accuracy: 0.5436
Epoch 61/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8598 - accuracy: 0.7787 - val_loss: 1.4596 - val_accuracy: 0.5369
Epoch 62/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8392 - accuracy: 0.7838 - val_loss: 1.4628 - val_accuracy: 0.5302
Epoch 63/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8285 - accuracy: 0.7855 - val_loss: 1.4542 - val_accuracy: 0.5503
Epoch 64/250
19/19 [==============================] - 0s 4ms/step - loss: 0.8115 - accuracy: 0.7905 - val_loss: 1.4549 - val_accuracy: 0.5369
Epoch 65/250
19/19 [==============================] - 0s 3ms/step - loss: 0.7996 - accuracy: 0.8007 - val_loss: 1.4444 - val_accuracy: 0.5302
Epoch 

19/19 [==============================] - 0s 3ms/step - loss: 0.4232 - accuracy: 0.8851 - val_loss: 1.4399 - val_accuracy: 0.5638
Epoch 117/250
19/19 [==============================] - 0s 4ms/step - loss: 0.4183 - accuracy: 0.8919 - val_loss: 1.4619 - val_accuracy: 0.5705
Epoch 118/250
19/19 [==============================] - 0s 3ms/step - loss: 0.4162 - accuracy: 0.8936 - val_loss: 1.4696 - val_accuracy: 0.5570
Epoch 119/250
19/19 [==============================] - 0s 5ms/step - loss: 0.4093 - accuracy: 0.8986 - val_loss: 1.4518 - val_accuracy: 0.5772
Epoch 120/250
19/19 [==============================] - 0s 4ms/step - loss: 0.4102 - accuracy: 0.8902 - val_loss: 1.4353 - val_accuracy: 0.5906
Epoch 121/250
19/19 [==============================] - 0s 4ms/step - loss: 0.4039 - accuracy: 0.8936 - val_loss: 1.4602 - val_accuracy: 0.5772
Epoch 122/250
19/19 [==============================] - 0s 5ms/step - loss: 0.4021 - accuracy: 0.9003 - val_loss: 1.4393 - val_accuracy: 0.5839
Epoch 123/250

19/19 [==============================] - 0s 3ms/step - loss: 0.2469 - accuracy: 0.9392 - val_loss: 1.6029 - val_accuracy: 0.5638
Epoch 174/250
19/19 [==============================] - 0s 4ms/step - loss: 0.2444 - accuracy: 0.9443 - val_loss: 1.5879 - val_accuracy: 0.5772
Epoch 175/250
19/19 [==============================] - 0s 5ms/step - loss: 0.2400 - accuracy: 0.9459 - val_loss: 1.5855 - val_accuracy: 0.5772
Epoch 176/250
19/19 [==============================] - 0s 3ms/step - loss: 0.2394 - accuracy: 0.9544 - val_loss: 1.6032 - val_accuracy: 0.5705
Epoch 177/250
19/19 [==============================] - 0s 4ms/step - loss: 0.2382 - accuracy: 0.9476 - val_loss: 1.6129 - val_accuracy: 0.5839
Epoch 178/250
19/19 [==============================] - 0s 4ms/step - loss: 0.2341 - accuracy: 0.9544 - val_loss: 1.6195 - val_accuracy: 0.5839
Epoch 179/250
19/19 [==============================] - 0s 4ms/step - loss: 0.2346 - accuracy: 0.9493 - val_loss: 1.6191 - val_accuracy: 0.5638
Epoch 180/250

19/19 [==============================] - 0s 4ms/step - loss: 0.1482 - accuracy: 0.9696 - val_loss: 1.7942 - val_accuracy: 0.5839
Epoch 231/250
19/19 [==============================] - 0s 3ms/step - loss: 0.1459 - accuracy: 0.9747 - val_loss: 1.7938 - val_accuracy: 0.5839
Epoch 232/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1432 - accuracy: 0.9696 - val_loss: 1.8007 - val_accuracy: 0.5772
Epoch 233/250
19/19 [==============================] - 0s 3ms/step - loss: 0.1438 - accuracy: 0.9713 - val_loss: 1.8047 - val_accuracy: 0.5906
Epoch 234/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1420 - accuracy: 0.9713 - val_loss: 1.7848 - val_accuracy: 0.5906
Epoch 235/250
19/19 [==============================] - 0s 3ms/step - loss: 0.1409 - accuracy: 0.9696 - val_loss: 1.7961 - val_accuracy: 0.5906
Epoch 236/250
19/19 [==============================] - 0s 4ms/step - loss: 0.1399 - accuracy: 0.9730 - val_loss: 1.8362 - val_accuracy: 0.5906
Epoch 237/250

In [36]:
loss,acc = model.evaluate(X_test_pca,Y_test_encoded)
loss,acc

10/10 [==============================] - 0s 2ms/step - loss: 1.8455 - accuracy: 0.6541


(1.845536708831787, 0.6540880799293518)